Install binary dependencies and required python modules:
* CMake
* CFFI
* Cython
* PyBind11

In [5]:
from sys import platform
import logging as log
import subprocess

log.basicConfig(format='[ %(levelname)s ] %(message)s', level=log.DEBUG)

log.info("Checking for installed CMake...")
p = subprocess.run(['which', 'cmake'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode == 0:
    log.info(f"CMake executable found at {p.stdout.decode()}")
else:
    log.warn("CMake executable not found! Installing ... ")
    cmd = ['brew', 'install', 'cmake'] if platform == 'darwin' else ['apt', 'install', 'cmake']
    log.info(f"Running: {' '.join(cmd)}...")
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.returncode != 0:
        log.error(f"Failed to install cmake.\n"
                  f"Process STDOUT:\n{p.stdout.decode()}\n"
                  f"Process STDERR:\n{p.stderr.decode()}")
    else:
        log.info("Installation succeed!")

log.info("Installing required python modules...")
p = subprocess.run(['pip', 'install', '-r', 'requirements.txt'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error(f"Failed to install required python modules.\n"
              f"Process STDOUT:\n{p.stdout.decode()}\n"
              f"Process STDERR:\n{p.stderr.decode()}")
else:
    log.info("Installation succeed")

[ INFO ] Checking for installed CMake...
[ INFO ] CMake executable found at /usr/local/bin/cmake

[ INFO ] Installing required python modules...
[ INFO ] Installation succeed


Bellow you can find C library functions declaration. For more convenience listings with each function implmentation
will be added bellow right before using it from python.

c_library/library.h
```c
//  CFFI doesn't support directives in headers
//  So for compatibility reason all includes moved to .c files
//#include <stdio.h>
//#include <string.h>
//#include <unistd.h>

int func_ret_int(int val);
double func_ret_double(double val);
char *func_ret_str(char *val);
char func_many_args(int val1, double val2, char val3, short val4);


void arr_minus_one(void *data, unsigned int arr_size, unsigned int el_size, char *el_type);
int *gen_static_arr();
void gen_arr(int *arr, unsigned int size);
void qsort_wrap(int *arr, unsigned int size, unsigned int el_size);
int comp (const int *a, const int *b);
```
Here is brief description of each method and its purpose:
* Functions demonstrating working with basic types. Do nothing besides of receiving arguments of certain types,
printing basic information, which will show us that real C function was invoked, and returning some value.
```c
int func_ret_int(int val);
double func_ret_double(double val);
char *func_ret_str(char *val);
char func_many_args(int val1, double val2, char val3, short val4);
```
* Functions demonstrating how to manipulate with arrays:
    *  `void arr_minus_one(void *data, unsigned int arr_size, unsigned int el_size, char *el_type);` - receives an
    untyped array `void *data`, elements count in array `unsigned int arr_size`,
    element size in bytes `unsigned int el_size` and string representation of array's element data type `char *el_type`.
    The function iterates over passed array and subtract 1 from each element without memory copying.
    * `int *gen_static_arr();` - generates static array with 10 integer sequential elements starting from 0 and returns
    pointer to generated array (Bad array generation approach)
    * `void gen_arr(int *arr, unsigned int size);` - fill pre-allocated integer array `int *arr` of given size
    `unsigned int size` with integer values in range `[0, size - 1]`
    * `void qsort_wrap(int *arr, unsigned int size, unsigned int el_size);` - wraps standard quick sort algorithm from libc.
    Will be used for performance comparison between C, Python and mixed quick sort implementation
    * `int comp (const int *a, const int *b)` - C implementation of comparison function required by standard qsort
    function from libc


Building C library from sources.

In [6]:
import os
import shutil

TUTOR_ROOT = os.getcwd()
CMAKE_ROOT_FOLDER = os.path.join(TUTOR_ROOT, 'c_library',)
BUILD_FOLDER = os.path.join(CMAKE_ROOT_FOLDER, 'build')
if os.path.exists(BUILD_FOLDER):
    shutil.rmtree(BUILD_FOLDER)
os.mkdir(BUILD_FOLDER)

os.chdir(BUILD_FOLDER)
log.info("Running CMake...")
p = subprocess.run(['cmake', os.path.pardir], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error("CMake run failed!\n"
              f"STDOUT:\n {p.stdout.decode()}\n"
              f"STDERR:\n {p.stderr.decode()}\n")
else:
    log.info("CMake files generation succeed!\n"
             f"STDOUT:\n {p.stdout.decode()}\n")

log.info("Running library building with make...")
p = subprocess.run(['make'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error("Building failed!\n"
              f"STDOUT:\n {p.stdout.decode()}\n"
              f"STDERR:\n {p.stderr.decode()}\n")
else:
    log.info("Building suceed succeed!\n"
             f"STDOUT:\n {p.stdout.decode()}\n")

os.chdir(TUTOR_ROOT)

if platform in ("linux", "linux2"):
    C_LIB_NAME = "libbindings_demo.so"
elif platform == "darwin":
    C_LIB_NAME = "libbindings_demo.dylib"
else:
    raise OSError(f"Unsupported platform {platform}")
C_LIB_PATH = os.path.join(BUILD_FOLDER, C_LIB_NAME)
if not os.path.exists(C_LIB_PATH):
    raise FileNotFoundError(f"Compiled library not found at {C_LIB_PATH}")
else:
    log.info(f"Compiled library located at {C_LIB_PATH}")

[ INFO ] Running CMake...
[ INFO ] CMake files generation succeed!
STDOUT:
 -- The C compiler identification is AppleClang 12.0.0.12000031
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/mikhailtreskin/repos/py_bindings_tutorial/c_library/build


[ INFO ] Running library building with make...
[ INFO ] Building suceed succeed!
STDOUT:
 Scanning dependencies of target bindings_demo
[ 25%] Building C object CMakeFiles/bindings_demo.dir/basic_funcs.c.o
[ 50%] Building C object CMakeFiles/bindings_demo.dir/arrays.c.o
[ 75%] Building C object CMakeFiles/bindings_demo.dir/vars.c.o
[100%] Linking C shared library libbindings_demo.dylib
[100%] Built target bindings_demo


[ INFO ] Compiled library located at /Users/mikhailtreskin/

**Ctypes demo**

[ctypes](https://docs.python.org/3/library/ctypes.html) is a foreign function library for Python. It provides C
compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

What's stands for foreign function library? It's a library which implements foreign function interface (FFI) convention
between two certain languages - between C as a host language and Python as a guest language in case of ctypes.
Briefly, FFI is an interface that allows code written in one language to call code written in another language.
Ctypes rely on [libffi](https://github.com/libffi/libffi) library implementing FFI calling convention for C as a host language.

Ctypes allows to make calls to C functions and refere to C objects from Python in a relatively easy way. Let's go
through several examples to on how to interact with precompiled C library from Python using ctypes

First of all we have to load dynamic linked library in a following way (on Windows WINDLL call should be used instead CDLL)

In [7]:
import ctypes

lib = ctypes.CDLL(C_LIB_PATH)

And now we have an access to all of the functions mentioned above and declared in `c_library/library.h`.
Let's start from calling a simple functions consuming one or several "simple" arguments.

*Functions listed bellow are defined in `c_library/basic_funcs.c`*

Working with integers on example of pretty simple C function consuming integer argument performing logging print from C
code and then returns same integer value, which was provided as an input argument:
```c
int func_ret_int(int val) {
    printf("From C Library: func_ret_int: %d\n", val);
    return val;
}

double func_ret_double(double val) {
    printf("From C Library: func_ret_double: %f\n", val);
    return val;
}

```

To call functions from C library we have to describe it's signature in Python by specifying input arguments types
and return value type of desired C function. And then we can call it as an usual Python method.

In [8]:
# Jupyter notebook suppress default output from our C library so custom manager to capture stdout from C was added
from stdout_capture import capture_c_stdout

# Specifying return type of the func_ret_int C function
lib.func_ret_int.restype = ctypes.c_int
# Specifying input arguments type of the func_ret_int C function
lib.func_ret_int.argtypes = [ctypes.c_int, ]

# Same for double data type
lib.func_ret_double.restype = ctypes.c_double
lib.func_ret_double.argtypes = [ctypes.c_double]
with capture_c_stdout():
    print('From Python Script: func_ret_int: ', lib.func_ret_int(101))
    print('From Python Script: func_ret_double: ', lib.func_ret_double(12.123456789))

From C Library: func_ret_int: 101
From Python Script: func_ret_int:  101
From C Library: func_ret_double: 12.123457
From Python Script: func_ret_double:  12.123456789


The same for char functions like:
```c
char * func_ret_str(char *val) {
    printf("From C Library: func_ret_str: %s\n", val);
    return val;
}
```

On Python level the only important aspects is that we have to encode input strings to bytearray and then also decode
return value

In [10]:
lib.func_ret_str.restype = ctypes.c_char_p
lib.func_ret_str.argtypes = [ctypes.c_char_p, ]
with capture_c_stdout():
    print('From Python Script: func_ret_str: ', lib.func_ret_str('Hello!'.encode('utf-8')).decode("utf-8"))

From C Library: func_ret_str: Hello!
From Python Script: func_ret_str:  Hello!


Also nothing outstanding for functions with several input arguments with different data types, e.g.:

```c
char func_many_args(int val1, double val2, char val3, short val4) {
    printf("From C Library: func_many_args: int - %d, double - %f, char - %c, short - %d\n", val1, val2, val3, val4);
    return val3;
}
```
To call the function all input argument types have to be defined in proper order:

In [11]:
lib.func_many_args.restype = ctypes.c_char
lib.func_many_args.argtypes = [ctypes.c_int, ctypes.c_double, ctypes.c_char, ctypes.c_short]
with capture_c_stdout():
    print('From Python Script: func_many_args: ',
      lib.func_many_args(15, 18.1617, 'X'.encode('utf-8'), 32000).decode("utf-8"))

From C Library: func_many_args: int - 15, double - 18.161700, char - X, short - 32000
From Python Script: func_many_args:  X


Ctypes also allows to access and modify variables from C library. In `c_library/vars.h` we have defined following variables
```c
int a = 5;
double b = 5.12345;
char c = 'X';
```

To access variables from loaded C library we need ro use `in_dll()` class method of corresponding data type of variable
we going to access. `in_dll()` requires two input arguments - the 1st is an loaded library handler, and the 2nd is string
with variable name. E.g.:

In [12]:
a = ctypes.c_int.in_dll(lib, "a")
print('From Python Script: a: ', a.value)
b = ctypes.c_double.in_dll(lib, "b")
print('From Python Script: b: ', b.value)
c = ctypes.c_char.in_dll(lib, "c")
print('From Python Script: c: ', c.value.decode("utf-8"))


From Python Script: a:  5
From Python Script: b:  5.12345
From Python Script: c:  X


Variables exported form a library can be modified in a following way

In [13]:
print(f"'a' initial value: {a.value}")
a.value = 22
a = ctypes.c_int.in_dll(lib, "a")
print(f"'a' modified value: {a.value}")

'a' initial value: 5
'a' modified value: 22


OK, it were a simple examples which barely will be used in real life and were provided as introduction and way
to understand how ctypes works. But what about real life examples?

Bellow we will have a look on how to have a deal with C arrays basing on manipulation with numpy arrays.
# TBA
# TBA
# TBA
# TBA